In [ ]:
#code for predicting intents from sarvam.ai for training and testing dataset


import requests
import pandas as pd
import time

# ----------------------------------------------
# Configuration
# ----------------------------------------------
INPUT_FILE             = "unseen_data.xlsx"        # Excel file with '001_pa' column
API_KEY                = "sk_zp6z696v_HnTVCWLL78Up5DjSod8whBxH"
OUTPUT_PREFIX          = "sarvam_unseen"         # Prefix for output files
BATCH_SIZE             = 16                             # Rows per batch
SAVE_EVERY_N_BATCHES   = 3                              # Save after this many batches (16×3 = 48 rows)
BACKOFF_SECONDS        = 30                             # Seconds to wait after any failed attempt
# ----------------------------------------------

# Load input data
df = pd.read_excel(INPUT_FILE, engine='openpyxl')
if "001_pa" not in df.columns:
    raise KeyError("Column '001_pa' not found in the input Excel file.")
df["intent"] = None

# A prompt that asks Sarvam.ai to extract intent freely (one or two Punjabi words),
# illustrating with a few varied examples without fixing a closed set of labels:
SYSTEM_PROMPT = """
ਤੁਸੀਂ ਇੱਕ ਪ੍ਰਸ਼ਿਖਤ intent classification ਮਾਡਲ ਹੋ।  
ਤੁਹਾਡਾ ਟਾਸਕ ਹੈ ਹਰੇਕ ਹੇਠ ਦਿੱਤੇ ਪੰਜਾਬੀ ਵਾਕ ਦਾ ਮੂਲ ਉਦੇਸ਼ (intent) ਨਿਰਧਾਰਤ ਕਰਨਾ — ਪੰਜਾਬੀ ਵਿੱਚ ਸਿਰਫ਼ 1 ਤੋਂ 4 ਸ਼ਬਦਾਂ ਵਿੱਚ।

👉 Intent ਦਾ ਅਰਥ ਹੈ: **ਇਸ ਵਾਕ ਦੇ ਪਿੱਛੇ ਲੁਕਿਆ ਹਵਾਲਾ ਜਾਂ ਉਦੇਸ਼ — ਲੇਖਕ/ਵਕਤਾ ਕਿਉਂ ਇਹ ਵਾਕ ਲਿਖ ਰਿਹਾ/ਬੋਲ ਰਿਹਾ ਹੈ?**  
ਉਦਾਹਰਨ:

- Greeting → ਸੁਆਗਤ
- Question → ਸਵਾਲ
- Request → ਬੇਨਤੀ
- Information → ਜਾਣਕਾਰੀ
- Emotion → ਭਾਵਨਾ
- Translation → ਅਨੁਵਾਦ
- Pronunciation → ਉਚਾਰਣ
- Travel Info → ਸੈਲਾਨੀ-ਸੂਚਨਾ
- Help → ਮਦਦ-ਨਿਵੇਦਨ
- Command → ਆਦੇਸ਼

**Examples**:

- ਸਤ ਸ੍ਰੀ ਅਕਾਲ! → ਸੁਆਗਤ  
- ਕੀ ਤੁਸੀਂ ਮੇਰੀ ਮਦਦ ਕਰ ਸਕਦੇ ਹੋ? → ਬੇਨਤੀ  
- ਮੈਂ ਦਫ਼ਤਰ ਜਾ ਰਿਹਾ ਹਾਂ। → ਜਾਣਕਾਰੀ  
- ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ? → ਸਵਾਲ  
- ਮੈਂ ਅੱਜ ਖੁਸ਼ ਹਾਂ। → ਭਾਵਨਾ  
- ਪਾਸਤਾ ਲਈ ਸਪੇਨੀ ਸ਼ਬਦ ਕੀ ਹੈ? → ਅਨੁਵਾਦ  
- ਚੀਨ ਵਿੱਚ ਸੈਲਾਨੀ ਆਕਰਸ਼ਣ ਕਿਹੜੇ ਹਨ? → ਸੈਲਾਨੀ-ਸੂਚਨਾ  
- ਇਹ ਸ਼ਬਦ ਕਿਵੇਂ ਉਚਾਰਿਆ ਜਾਂਦਾ ਹੈ? → ਉਚਾਰਣ  

**Guidelines**:

✅ ਹਰ ਵਾਕ ਲਈ ਸਿਰਫ਼ intent ਲਿਖੋ — 1-4 ਸ਼ਬਦ (ਪੰਜਾਬੀ ਵਿੱਚ)  
✅ Sentence ਨਾ ਦੁਹਰਾਓ। "intent:" ਨਾ ਜੋੜੋ।  
✅ Similar ਵਾਕਾਂ ਲਈ same intent wording ਵਰਤੋ।  
✅ ਜੇ sentence ਨਵਾਂ ਹੈ → ਅਰਥ ਪੜ੍ਹੋ, ਸਹੀ intent ਲਿਖੋ।  
✅ ਸੰਦੇਹਪੂਰਨ ਹੋਣ 'ਤੇ → ਸਭ ਤੋਂ ਘੱਟ ਸਮਝ ਆਉਣ ਵਾਲਾ ਪਰ ਲਾਗੂ intent ਚੁਣੋ।  
✅ ਖਾਲੀ ਜਾਂ ਵਿਅਰਥ ਉੱਤਰ ਨਾ ਦਿਓ।  
✅ Output ਸਿਰਫ਼ intent ਹੀ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ।

**Thinking step**:

ਪਹਿਲਾਂ, ਹਰ sentence ਨੂੰ ਆਪਣੇ ਮਨ ਵਿੱਚ ਪੜ੍ਹੋ ਅਤੇ ਸੋਚੋ:

👉 "ਇਹ sentence ਕਿਹੜੀ category ਵਿੱਚ ਪੈਂਦਾ ਹੈ?"  
👉 "ਲੇਖਕ/ਵਕਤਾ ਦਾ ਉਦੇਸ਼ ਕੀ ਹੈ?"  
👉 "ਕੀ ਇਹ ਬੇਨਤੀ ਹੈ? ਸਵਾਲ? ਜਾਣਕਾਰੀ? ਭਾਵਨਾ? ਹੋਰ?"

ਫਿਰ ਸਿਰਫ਼ intent ਲਿਖੋ — 1 line per sentence.

ਹੁਣ ਹੇਠਾਂ ਦਿੱਤੇ ਵਾਕਾਂ ਦੇ intent ਲਿਖੋ:
""".strip()


def get_intent_from_sarvam(text, api_key):
    url = "https://api.sarvam.ai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "api-subscription-key": api_key
    }
    payload = {
        "model": "sarvam-m",
        "messages": [
            {"role": "system",  "content": SYSTEM_PROMPT},
            {"role": "user",    "content": text}
        ]
    }
    try:
        resp = requests.post(url, json=payload, headers=headers)
        if resp.status_code != 200:
            print(f"[Error] API call failed {resp.status_code}: {resp.text}")
            return None
        data = resp.json()
        choices = data.get("choices", [])
        if not choices:
            return None
        content = choices[0].get("message", {}).get("content", "").strip()
        # Take at most two words from the start (in case of extra whitespace/punctuation)
        words = content.split()
        return " ".join(words[:2]) if words else None
    except Exception as e:
        print(f"[Exception] during API call: {e}")
        return None

# Batch-processing loop
total_rows   = len(df)
num_batches  = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE
file_counter = 1

sent_buffer   = []
intent_buffer = []

for batch_idx in range(num_batches):
    start = batch_idx * BATCH_SIZE
    end   = min(start + BATCH_SIZE, total_rows)
    print(f"\n🔄 Batch {batch_idx+1}/{num_batches} (rows {start}–{end-1})")

    for row_idx in range(start, end):
        text   = df.at[row_idx, "001_pa"]
        intent = None

        # Retry until success
        while intent is None:
            intent = get_intent_from_sarvam(text, API_KEY)
            if intent is None:
                print(f"  ⚠️ Failed to get intent for row {row_idx}, sleeping {BACKOFF_SECONDS}s…")
                time.sleep(BACKOFF_SECONDS)
            else:
                print(f"  ✅ Row {row_idx}: intent = '{intent}'")

        df.at[row_idx, "intent"] = intent
        sent_buffer.append(text)
        intent_buffer.append(intent)

        # short pause on success
        time.sleep(1)

    # After each batch, save if needed
    batches_done = batch_idx + 1
    if (batches_done % SAVE_EVERY_N_BATCHES == 0) or (end == total_rows):
        out_df   = pd.DataFrame({"Sentence": sent_buffer, "Intent": intent_buffer})
        out_path = f"{OUTPUT_PREFIX}_part{file_counter}.xlsx"
        out_df.to_excel(out_path, index=False)
        print(f"📦 Saved {len(out_df)} rows → {out_path}")

        sent_buffer.clear()
        intent_buffer.clear()
        file_counter += 1

print("\n🎉 All batches finished and saved!")



🔄 Batch 1/13 (rows 0–15)
  ✅ Row 0: intent = 'ਜਾਣਕਾਰੀ'
  ✅ Row 1: intent = 'ਬੇਨਤੀ'
  ✅ Row 2: intent = 'ਜਾਣਕਾਰੀ'
  ✅ Row 3: intent = 'ਜਾਣਕਾਰੀ'
  ✅ Row 4: intent = 'ਬੇਨਤੀ'
  ✅ Row 5: intent = 'ਸੋਚ'
  ✅ Row 6: intent = 'ਸਵਾਲ'
  ✅ Row 7: intent = 'ਜਾਣਕਾਰੀ'
  ✅ Row 8: intent = 'ਮੈਂ ਕੱਲ੍ਹ'
  ✅ Row 9: intent = 'ਭਾਵਨਾ'
  ✅ Row 10: intent = 'ਅਨੁਵਾਦ'
  ✅ Row 11: intent = 'ਆਦੇਸ਼'
  ✅ Row 12: intent = 'ਸੈਲਾਨੀ-ਸੂਚਨਾ'
  ✅ Row 13: intent = 'ਜਾਣਕਾਰੀ'
  ✅ Row 14: intent = 'ਸੁਆਗਤ'
[Error] API call failed 429: {"error":{"message":"Rate limit exceeded","code":"rate_limit_exceeded_error","request_id":"20250624_28b5240c-f30c-4d37-9f08-37447bb0c98b"}}
  ⚠️ Failed to get intent for row 15, sleeping 30s…
[Error] API call failed 429: {"error":{"message":"Rate limit exceeded","code":"rate_limit_exceeded_error","request_id":"20250624_2bef467e-4648-4d53-bddd-892773014b3d"}}
  ⚠️ Failed to get intent for row 15, sleeping 30s…
  ✅ Row 15: intent = 'ਬੇਨਤੀ'

🔄 Batch 2/13 (rows 16–31)
  ✅ Row 16: intent = 'ਅਨੁਵਾਦ'
  